In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
import tensorflow.math as tfmath
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc

2024-01-11 22:16:03.910438: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-11 22:16:03.978417: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class DNN_VAE_Model(Model):
    def __init__(self,input_dim, h_dim_1, h_dim_2, latent_dim):
        super(DNN_VAE_Model,self).__init__()
        self.input_dim=input_dim
        self.e1=layers.Dense(h_dim_1, activation='relu')
        self.e2=layers.Dense(h_dim_2, activation='relu')
        self.mean=layers.Dense(latent_dim, activation='relu')
        self.logvar=layers.Dense(latent_dim, activation='relu')

        self.d1=layers.Dense(h_dim_2, activation='relu')
        self.d2=layers.Dense(h_dim_1, activation='relu')
        self.d3=layers.Dense(input_dim)
        
    def sample(self,m,logvar):
        std=tfmath.exp(0.5*logvar)
        eps=tf.random.normal(tf.shape(m))
        x=m+eps*std
        return x
    
    def call(self, inputs):
        x= self.e1(inputs)
        x= self.e2(x)
        m=self.mean(x)
        logvar=self.logvar(x)
        self.m=m
        self.lv=logvar

        x=self.sample(m,logvar)

        x=self.d1(x)
        x=self.d2(x)
        z=self.d3(x)
        return z
    
    def loss_function(self,y_true,y_pred,beta=1):
        """masked mse"""
        mask = K.cast(K.not_equal(y_true, 0), K.floatx())
        squared_difference = K.square(mask * (y_pred - y_true))
        L1= K.mean(squared_difference)

        mu=self.m
        logv=self.lv

        KL=-0.5*K.mean(1+logv-mu**2 - tfmath.exp(logv))

        loss=L1+beta*KL
        return loss




def loss_fn(y_true, y_pred):
    """masked mse"""
    mask = K.cast(K.not_equal(y_true, 0), K.floatx())
    squared_difference = K.square(mask * (y_pred - y_true))
    return K.mean(squared_difference)

class KLDivergenceLayer(keras.layers.Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs,beta):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(beta*K.mean(kl_batch), inputs=inputs)

        return inputs

In [3]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

truthtable=[]

threshold=50
for i, batch in enumerate(Dataset):
  if np.sum(batch[:,0])>=threshold:
    truthtable+=[1]
  else:
    truthtable+=[0]

event_pt_br=[]
Data_Test_full=Dataset[2000001:3600000,:,:]
for j, br_1 in enumerate(Data_Test_full):
  event_pt_br+=[np.sum(br_1[:,0])]

for i, batch in enumerate(Dataset):
  pt_sum=0
  for j, particle in enumerate(Dataset[i,:,:]):
    if particle[3]!=0:
      pt_sum+=particle[0]
  for j, particle in enumerate(Dataset[i,:,:]):
    particle[0]=particle[0]/pt_sum

Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Test_Truth=truthtable[2000001:3600000]
Data_Validate=Dataset[3600001:4000000,:,0:3]

In [4]:
Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [46]:
input_dim = 57
h_dim_1 = 32
h_dim_2 = 16
latent_dim = 8
beta=1
inputs=keras.Input(shape=(input_dim,))
x=layers.Dense(h_dim_1, activation='relu')(inputs)
x=layers.Dense(h_dim_2, activation='relu')(x)
m=layers.Dense(latent_dim, activation='relu')(x)
logvar=layers.Dense(latent_dim, activation='relu')(x)

def sample(args):
    m,logvar=args
    std=tfmath.exp(0.5*logvar)
    eps=tf.random.normal(tf.shape(m))
    x=m+eps*std
    return x

m,logvar=KLDivergenceLayer()([m,logvar],1)

x=layers.Lambda(sample,output_shape=(input_dim,))([m,logvar])
x=layers.Dense(h_dim_2, activation='relu')(x)
x=layers.Dense(h_dim_1, activation='relu')(x)
outputs=layers.Dense(input_dim)(x)

DNN_VAE = keras.Model(inputs=inputs, outputs=outputs, name="DNN_VAE")
DNN_VAE.summary()

Model: "DNN_VAE"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 57)]         0           []                               
                                                                                                  
 dense_133 (Dense)              (None, 32)           1856        ['input_17[0][0]']               
                                                                                                  
 dense_134 (Dense)              (None, 16)           528         ['dense_133[0][0]']              
                                                                                                  
 dense_135 (Dense)              (None, 8)            136         ['dense_134[0][0]']              
                                                                                            

In [5]:
input_dim = 57
h_dim_1 = 32
h_dim_2 = 16
latent_dim = 8
beta=1
inputs=keras.Input(shape=(input_dim,))
x=layers.Dense(h_dim_1, activation='relu')(inputs)
x=layers.Dense(h_dim_2, activation='relu')(x)
z_mean=layers.Dense(latent_dim, activation='relu')(x)
z_logvar=layers.Dense(latent_dim, activation='relu')(x)

def sample(args):
    m,logvar=args
    std=tfmath.exp(0.5*logvar)
    eps=tf.random.normal(tf.shape(m))
    x=m+eps*std
    return x

z=layers.Lambda(sample)([z_mean,z_logvar])

encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')

latent_inputs=keras.Input(shape=(latent_dim,),name='z_sampling')
x=layers.Dense(h_dim_2, activation='relu')(latent_inputs)
x=layers.Dense(h_dim_1, activation='relu')(x)
outputs=layers.Dense(input_dim)(x)
decoder=keras.Model(latent_inputs,outputs,name='decoder')

outputs=decoder(encoder(inputs)[2])
vae=keras.Model(inputs,outputs,name='vae')
vae.summary()
def loss_fn(y_true, y_pred):
    """masked mse"""
    mask = K.cast(K.not_equal(y_true, 0), K.floatx())
    squared_difference = K.square(mask * (y_pred - y_true))
    return K.mean(squared_difference)

def KL_func(mu,logv):
    KL=-0.5*K.mean(1+logv-mu**2 - tfmath.exp(logv))
    loss=KL
    return loss
    
recon_loss=loss_fn(inputs,outputs)
KL_loss=KL_func(z_mean,z_logvar)
vae_loss=recon_loss+beta*KL_loss
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 50
BATCH_SIZE = 1024

history = vae.fit(x=Data_Train_Flat, y=Data_Train_Flat, validation_data=(Data_Val_Flat,Data_Val_Flat), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks)


Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 57)]              0         
                                                                 
 encoder (Functional)        [(None, 8),               2656      
                              (None, 8),                         
                              (None, 8)]                         
                                                                 
 decoder (Functional)        (None, 57)                2569      
                                                                 
Total params: 5,225
Trainable params: 5,225
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1954/1954 [==============================] - 17s 7ms/step - loss: 0.2164 - val_loss: 0.2154 - lr: 0.0010
Epoch 2/50
1954/1954 [==============================] - 13s 7ms/step - loss: 

In [7]:
vae.save('/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/40MHZ_norm_DNN_VAE.keras',overwrite=True)

In [9]:


vae2=keras.saving.load_model('/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/40MHZ_norm_DNN_VAE.keras',compile=True)

In [32]:
class LossHistory(keras.callbacks.Callback):
    def on_test_begin(self, logs={}):
        self.losses = []

    def on_test_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history=LossHistory()
vae2.evaluate(Data_Val_Flat,batch_size=1,callbacks=[history])
print(history.losses)

 25586/399999 [>.............................] - ETA: 10:59 - loss: 0.2153

KeyboardInterrupt: 